## Проектирование системы водоснабжения
Рассматривается задача проектирования системы водоснабжения в пустынной местности. Система состоит из солнечной батареи, опреснительной установки, резервуара для воды. При нехватке воды есть возможность ее доставить.

Активность Солнца полагается случайной, следовательно, выработка пресной воды в установке также случайна

### Обозначения
Пусть

$d_j$ - заданное потребление воды в j-й месяц

$u_j$ - объем внешне доставленной воды

$s$ - площадь солнечной батареи

$v$ - вместимость резервуара

$X_j$ - производительность солнечной батареи

Суммарная стоимость системы определяется соотношением
$$
\Phi(u) \triangleq a_1s + a_2v + a_0\sum\limits_{j=1}^tu_i,
$$
где $a_0$ - стоимость доставки одного кубометра воды,  $a_1$ - стоимость квадратного метра солнечной батареи, $a_2$ - стоимость одного кубометра резервуара. В расчетах использовались значения $a_0=250, a_1=37.5, a_2 = 100$.

Значения остальных параметров приведены в коде

### Ограничения
Будем считать, что система работает нормально, если она покрывает потребность в воде с заданной вероятностью.
$$
P(Z_j(u, X)\ge 0, j=\overline{1,t}) \ge \alpha
$$
$Z_j$ - остаток воды в j-м месяце

\begin{gather}
Z_1(u, X) = X_1s + u_1 - d_1,\\
Z_j(u, X) = \min\{Z_{j-1}, v\} + X_js + u_j - d_j
\end{gather}

Исходное вероятностное ограничение перепишем в виде
$$
P(Z_j(u, X)\ge 0, j=\overline{1,t}) = P\left(\min\limits_{j=\overline{1,t}}Z_j(u, X)\ge 0\right)
$$
Минимум в расчетах будет заменен на гладкий минимум, для которого может быть вычислена производная.

# Итоговая задача оптимизации
$$
\Phi(u) \to \min
$$
при ограничении
$$
P\left(\min\limits_{j=\overline{1,t}}Z_j(u, X)\ge 0\right) \ge \alpha
$$
Временной горизонт в исходниках - переменная величина в месяцах. Может принимать значения от 2 до 6.

In [8]:
from LFlow.stochastic_model import IndependentGenerator, MultivariateGaussGenerator, stats
from LFlow.labos_flow_v2 import LabFunc, np, plt, Point, LabSigmoid, LabSmoothmin, Identity
# параметры
alpha = 0.99
max_iter = 200
step = 0.01
t = 6
sample_size = 3000
theta = 10
# потребление воды по месяцам
ds = np.array([29.6, 23.9, 36.2, 82.1, 96.5, 173.4])
# математические ожидания для случайных величин активности солнца
ms = np.array([0.0084, 0.0083, 0.0185, 0.063, 0.123, 0.137])
sigmas = np.array([5.8, 5.5, 12.3, 42.1, 81.8, 91.6])*1e-4
a0 = 250
a1 = 37.5
a2 = 100

In [9]:
# setup
loss_fun = '{}*ss + {}*vv + {}*({})'.format(a1, a2, a0, '+'.join(['u{}'.format(i) for i in range(1, t+1)]))
loss_grad = {'ss' : a1, 'vv' : a2}
for i in range(1, t+1):
    loss_grad['u{}'.format(i)] = a0
loss_grad['a0'] = '+'.join([f'u{i}' for i in range(1, t+1)])

loss = LabFunc(loss_fun, loss_grad)
v = Identity('vv')
args = ['ss', 'vv'] + [f'u{i}' for i in range(1, t+1)]
u_args = args[2:]

dd = ds[0]
Zs = {1 : LabFunc(f'X1*ss + u1 - {dd}', {'ss' : 'X1', 'u1' : 1, 'vv' : 0, 'X1' : 'ss'})}
#conditions = [LabSigmoid(Zs[1], theta=7)]
for i in range(1, t):
    """
    собираем ограничения на Zj = min{Zj-1, v} + sXj + uj - dj
    """
    dd = ds[i]
    Zs[i+1] = LabSmoothmin(Zs[i], v, theta=theta) + LabFunc(f'ss*X{i+1} + u{i+1} - {dd}', {'ss' : f'X{i+1}', 'vv' : 0, f'u{i+1}' : 1, f'X{i+1}' : 'ss'})
    #conditions.append(LabSigmoid(Zs[i+1], theta=7))

if len(Zs.keys()) > 1:
    total_cond = LabSigmoid(LabSmoothmin(*Zs.values(), theta=theta), theta=theta)
else:
    total_cond = LabSigmoid(Zs[1], theta=theta)

In [10]:
"""
собираем генератор случайностей:)
независимые гауссовские величины
параметры для каждого свои
"""
variables = []
models = []
for i in range(t):
    variables.append(f'X{i+1}')
    models.append(stats.norm(loc=ms[i], scale=sigmas[i]))
    
sm = IndependentGenerator(variables, models)
points, sample = sm.rvs(sample_size)

In [11]:
"""
используем самопальный адаптивный градиентный спуск, проверяя на пробой ограничений
когда упираемся в ограничение считаем проекцию градиента
"""
start_point = Point({'ss' : 2500, 'vv' : 66, 'u1' : 30})
for i in range(2, t+1):
    start_point = start_point.expand({f'u{i}' : 16})

cond_mean = sm.papa_carlo(total_cond, start_point, sample)
while cond_mean < alpha:
    start_point['ss'] += 1
    start_point['vv'] += 1
    start_point['u1'] += 1
    cond_mean = sm.papa_carlo(total_cond, start_point, sample)

print('Значение вероятности в стартовой точке {:.3f}'.format(cond_mean))

def bound_point(point, direction):
    """
    вспомогательная функция для проверки ограничений на пробой
    учитывает только ограничения на неотрицательность переменных
    возвращает новую точку
    """
    res = Point(point.dict.copy())
    direction = Point(direction.dict.copy())
    factor = 1
    for arg in args:
        if (point[arg] == 0) and (direction[arg] < 0):
            direction[arg] = 0
        elif point[arg] + direction[arg] < 0:
            factor = min(factor, abs(point[arg]/direction[arg]))
    res = res + factor*direction
    return res
        
            
    

opt_crit = loss(start_point)
opt_point = Point(start_point.dict)
start_factor = 1
"""
собираем изменения стоимости за последние 10 шагов
прерываем цикл если среднее изменение оказывается меньше 1 у.е.
"""
crit_tail = [opt_crit]
for j in range(max_iter):
    loss_grad = loss.deriv(start_point).shrink(args)
    factor = start_factor
    """
    проверяем на пробой ограничений по u
    """
    cond_mean = sm.papa_carlo(total_cond, bound_point(start_point,  - factor*step*loss_grad), sample)
    if cond_mean < alpha:
        """
        потихоньку уменьшаем начальный шаг
        """
        start_factor = 0.8*start_factor
        cond_mean, cond_grad = sm.papa_carlo(total_cond, bound_point(start_point, -0.8*factor*step*loss_grad), sample, derivs=args)
        if cond_mean < alpha:
            """
            уменьшение шага не помогает остаться в допустимой области
            делаем шаг назад и считаем проекцию градиента
            """
            direction = loss_grad - cond_grad*cond_grad.cos(loss_grad)*loss_grad.norm()/cond_grad.norm()
            inner_factor = 1
            cond_mean, cond_grad = sm.papa_carlo(total_cond, bound_point(start_point, -step*direction), sample, derivs=args)
            while cond_mean < alpha:
                """
                если после шага вдоль ограничений попали вне допустимой области
                """
                inner_factor *= 0.75
                cond_mean = sm.papa_carlo(total_cond, bound_point(start_point, -inner_factor*step*direction), sample)
            start_point = bound_point(start_point, -inner_factor*step*direction)
        else:
            start_point = bound_point(start_point, -0.8*factor*step*loss_grad)
    else:
        start_point = bound_point(start_point, -factor*step*loss_grad)
    crit = loss(start_point)
    if crit < opt_crit:
        opt_crit = crit
        opt_point = Point(start_point.dict.copy())
    print('Price {:.1f} s {:.1f} v {:.1f} u [{}] prob {:.3f}'.format(crit, start_point['ss'], start_point['vv'],
          ', '.join(['{:.1f}'.format(start_point[u_arg]) for u_arg in u_args]), cond_mean))
    
    crit_tail.append(crit)
    if len(crit_tail) > 10:
        crit_tail = crit_tail[1:]
    if np.mean(np.abs(np.diff(crit_tail))) < 1:
        break
print('Optimal solution')
print('Price {:.1f} s {:.1f} v {:.1f} u [{}]'.format(opt_crit, opt_point['ss'], opt_point['vv'],
          ', '.join(['{:.1f}'.format(opt_point[u_arg]) for u_arg in u_args])))

Значение вероятности в стартовой точке 1.000
Price 123985.9 s 2499.6 v 65.0 u [27.5, 13.5, 13.5, 13.5, 13.5, 13.5] prob 1.000
Price 120121.9 s 2499.2 v 64.0 u [25.0, 11.0, 11.0, 11.0, 11.0, 11.0] prob 1.000
Price 116257.8 s 2498.9 v 63.0 u [22.5, 8.5, 8.5, 8.5, 8.5, 8.5] prob 1.000
Price 112393.8 s 2498.5 v 62.0 u [20.0, 6.0, 6.0, 6.0, 6.0, 6.0] prob 1.000
Price 108529.7 s 2498.1 v 61.0 u [17.5, 3.5, 3.5, 3.5, 3.5, 3.5] prob 1.000
Price 105438.4 s 2497.8 v 60.2 u [15.5, 1.5, 1.5, 1.5, 1.5, 1.5] prob 0.995
Price 103871.7 s 2497.6 v 59.6 u [15.5, 1.5, 0.0, 0.0, 0.0, 0.0] prob 0.995
Price 103871.7 s 2497.6 v 59.6 u [15.5, 1.5, 0.0, 0.0, 0.0, 0.0] prob 0.995
Price 103760.4 s 2497.3 v 58.6 u [15.5, 1.5, 0.0, 0.0, 0.0, 0.0] prob 0.995
Price 103649.1 s 2497.0 v 57.6 u [15.5, 1.5, 0.0, 0.0, 0.0, 0.0] prob 0.995
Price 103537.8 s 2496.6 v 56.6 u [15.5, 1.5, 0.0, 0.0, 0.0, 0.0] prob 0.995
Price 103426.4 s 2496.3 v 55.6 u [15.5, 1.5, 0.0, 0.0, 0.0, 0.0] prob 0.995
Price 103315.0 s 2495.9 v 54.6 u 